In [1]:
import cv2
import torch
import numpy as np


# YOLOv5 modelini yuklaymiz
model_path = 'yolov5s.pt'

device = "cpu"  # cpu uchun
# device = 0  #  gpu uchun
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)  # local model
model.to(device)

# video yuklaymiz
video = cv2.VideoCapture('trimmedVideo_11.mp4')


width, height = 1280, 720
fps = video.get(cv2.CAP_PROP_FPS)

# Qayta ishlangan videoni saqlash uchun VideoWriter ob'ektini yarating
_output1 = cv2.VideoWriter('_output_ABS.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Ko'pburchak ROIni aniqlang (Region of intersets)
roi_points = np.array([[400, 720], [400, 400], [870, 400], [870, 720]], np.int32)

#Ob'ektning haqiqiy o'lchamlarini aniqlaymiz
object_width = 50  # in centimeters

# Kameraning fokus uzunligini kiritamiz.
focal_length = 1000  # in pixels

# CWS uchun masofa oralig'i
dist_ = 12

# Videoning har bir kadrini qayta ishlash
while True:
    # Read the next frame
    success, frame = video.read()
    frame = cv2.resize(frame, (1280, 720))
    if not success:
        break

    # Ko'pburchak ROIni chizish
    cv2.polylines(frame, [roi_points], True, (0, 200, 0), 2)

    # ROI ichidagi uchta gorizontal chiziqning y-koordinatalarini hisoblash
    line_y1 = 600
    line_gap = 10
    line_ys = [line_y1 + i * line_gap for i in range(dist_)]

    # Chiziq ranglarini va kesishgan chiziq sonini boshlang
    line_colors = [(255, 0, 0) for _ in range(dist_)]
    crossed_lines = []

    # Kadrda ob'ektni aniqlashni amalga oshirish
    results = model(frame, size=320)
    detections = results.pred[0]

    # Ramkaning o'rtasida bo'linish chizig'ini tortish
    height, width, _ = frame.shape
    cv2.line(frame, (width // 2, 0), (width // 2, height), (160, 160, 160), 2)

    # Chegaraning markazlari ROI ichida yoki yo'qligini tekshirish
    for detection in detections:
        xmin = detection[0]
        ymin = detection[1]
        xmax = detection[2]
        ymax = detection[3]
        score = detection[4]
        class_id = detection[5]


        # Chegara ball
        if score >= 0.4:

            # Markaz koordinatalarini hisoblash
            if xmin < frame.shape[1] // 2:
                # Ramkaning chap tomoni
                centroid_x = int(xmax)
                centroid_y = int(ymax)
            else:
                # Ramkaning o'ng tomoni
                centroid_x = int(xmin)
                centroid_y = int(ymax)

            # Chegaraning markazlari ROI ichida yoki yo'qligini tekshirish
            if cv2.pointPolygonTest(roi_points, (centroid_x, centroid_y), False) > 0:
                cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 255, 0), -1)

                    # Chegara qutisi biron bir chiziqqa tegib yoki yo'qligini tekshirish
                for i, line_y in enumerate(line_ys):
                    if ymax >= line_y:
                        line_colors[i] = (0, 0, 255)  # Chiziq rangini qizil rangga o'zgartirish
                        crossed_lines.append(i + 1)  # Ro'yxatga kesishgan chiziq raqamini qo'shish

                # Ob'ektgacha bo'lgan masofani hisoblash
                distance = (object_width * focal_length) / (xmax - xmin)

                # Chegara ramkasini chizish va masofani ko'rsatish
                cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 0, 255), 2)
                cv2.putText(frame, f"Masofa: {distance:.2f} cm", (int(xmin), int(ymin) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 255), 1)
            else:
                cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)



    for line_y, color in zip(line_ys, line_colors):
        cv2.line(frame, (roi_points[0][0], line_y), (roi_points[2][0], line_y), color, 2)

    # Kesilgan chiziq raqamlarini chop etish
    if crossed_lines:
        cv2.putText(frame, 'Tormoz', (1000- 25, 100 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        crossed_lines_str = ', '.join(str(line_num) for line_num in crossed_lines)
        cv2.putText(frame, str(max(crossed_lines)), (1000 , 100 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)


        if max(crossed_lines) == 4 and max(crossed_lines) <= 5:
            cv2.putText(frame, 'OLDINDA To\'qnashuv ogohlantirish', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
        elif max(crossed_lines) == 6 and max(crossed_lines) <= 8:
            cv2.putText(frame, 'To\'qnashuv haqida OGOHLANTIRISH', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
        elif 9 <= max(crossed_lines) <= 11:
            cv2.putText(frame, 'DIQQAT BERING va NAZORATGA OLING', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
        elif max(crossed_lines) >= 11:
            cv2.putText(frame, 'Favqulodda To\'xtash ..!!', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)



    # Ramkani ko'rsatish
    cv2.imshow("Video", frame)

    # Chiqarilgan videoga ishlov berilgan kadrni yozish
    _output1.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Videoga olish va yozish ob'ektlarini qo'yib yuborish
video.release()
_output1.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\UzDev/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-11-26 Python-3.10.9 torch-2.1.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AssertionError: Torch not compiled with CUDA enabled